Install dependencies and set base URL's

In [21]:
%%capture
!pip install gradio

In [22]:
import requests
import json
import uuid
import datetime
import time
import gradio as gr

from google.colab import userdata


BASE_URL = "https://orgfarm-82bd55ac7d-dev-ed.develop.my.salesforce.com"
AGENT_API_BASE_URL = "https://api.salesforce.com/einstein/ai-agent/v1"

Generate Random UUID

In [23]:
# Generate a random UUID
def generate_random_uuid():
    return str(uuid.uuid4())

Generate Timestamp

In [24]:
def get_current_timestamp():
    return int(time.time())

Authenticate and get access token

In [50]:
def authenticate():
    payload = {
        'grant_type': 'client_credentials',
        'client_id': userdata.get('CLIENT_ID'),
        'client_secret': userdata.get('CLIENT_SECRET')
        #'username': 'userdata.get('USERNAME')',
        #'password': 'userdata.get('PASSWORD')'
    }
    response = requests.post(f"{BASE_URL}/services/oauth2/token", data=payload)
    if response.status_code == 200:
        access_token = response.json().get("access_token")
        return access_token
    else:
        return None

Start a Agent session

In [53]:
def start_session():
    global global_session_id, global_access_token
    access_token = authenticate();
    if not access_token:
        return "Authentication failed!"

    global_access_token = access_token;

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    session_payload = {
        "externalSessionKey": generate_random_uuid(),
        "instanceConfig": {
            "endpoint": BASE_URL
        },
        "variables": [],
        "streamingCapabilities": {
            "chunkTypes": ["Text"]
        },
        "bypassUser": 'false'
    }
    response = requests.post(f"{AGENT_API_BASE_URL}/agents/{userdata.get('AGENT_ID')}/sessions", headers=headers, json=session_payload)
    print(response.text)
    if response.status_code == 200:
        session_id = response.json().get("sessionId")
        global_session_id = session_id;
        return "Session started successfully!"
    else:
        return "Failed to start session!"
start_session()

{"sessionId":"15a614c2-66e2-4d27-b409-ae8ad9a07212","_links":{"self":null,"messages":{"href":"https://api.salesforce.com/einstein/ai-agent/v1/sessions/15a614c2-66e2-4d27-b409-ae8ad9a07212/messages"},"messagesStream":{"href":"https://api.salesforce.com/einstein/ai-agent/v1/sessions/15a614c2-66e2-4d27-b409-ae8ad9a07212/messages/stream"},"session":{"href":"https://api.salesforce.com/einstein/ai-agent/v1/agents/0XxgK000000OrhVSAS/sessions"},"end":{"href":"https://api.salesforce.com/einstein/ai-agent/v1/sessions/15a614c2-66e2-4d27-b409-ae8ad9a07212"}},"messages":[{"type":"Inform","id":"9bd800de-97ba-4a44-a37c-87dd53149c91","metrics":{},"feedbackId":"","planId":"","isContentSafe":true,"message":"Hi, I’m Agentforce! I use AI to search trusted sources, and more. Ask me “What else can you do?” to see how I can simplify your workday. How can I help?","result":[],"citedReferences":[]}]}


'Session started successfully!'

Send synchronous message to Agent

In [54]:
def send_synchronous_message(message):

    headers = {
        "Authorization": f"Bearer {global_access_token}",
        "Content-Type": "application/json"
    }

    message_payload = {
        "message": {
            "sequenceId": get_current_timestamp(),
            "type": "Text",
            "text": message
          },
        "variables": []
    }

    response = requests.post(f"{AGENT_API_BASE_URL}/sessions/{global_session_id}/messages", headers=headers, json=message_payload)

    if response.status_code == 200:
      message_data = response.json();
      return message_data["messages"][0]["message"]
    else:
      return "Failed to send synchronous message!"

Send message stream to Agent

In [55]:
def send_message_stream(message):

    headers = {
        "Authorization": f"Bearer {global_access_token}",
        "Content-Type": "application/json"
    }

    message_payload = {
        "message": {
          "sequenceId": get_current_timestamp(),
          "type": "Text",
          "text": message
        },
      "variables": []
    }

    response = requests.post(f"{AGENT_API_BASE_URL}/sessions/{global_session_id}/messages/stream", headers=headers, json=message_payload, stream=True)
    result = ""
    if response.status_code == 200:
        for chunk in response.iter_content(chunk_size=None):
            if chunk:
                result += chunk.decode("utf-8") + "\n"
    else:
        result = "Failed to send message stream!"

    return result

End Session

In [56]:
def end_session():
    headers = {
        "Authorization": f"Bearer {global_access_token}",
        "x-session-end-reason": "UserRequest"
    }
    response = requests.delete(f"{AGENT_API_BASE_URL}/sessions/{global_session_id}", headers=headers)
    if response.status_code == 200:
        global_session_id = None
        global_access_token = None
        return "Session ended successfully!"
    else:
        return "Failed to end session!"

Gradio Interface for Synchronous Messages

In [58]:
def gradio_sync_ui():
    with gr.Blocks() as sync_demo:
        gr.Markdown("## SF Headless Agent API - Sync Message")
        start_button = gr.Button("Start Session")
        input_text = gr.Textbox(label="Enter your message")
        sync_button = gr.Button("Send Sync Message")
        output_text = gr.Textbox(label="Response")
        end_button = gr.Button("End Session")

        start_button.click(start_session, outputs=output_text)
        sync_button.click(send_synchronous_message, inputs=input_text, outputs=output_text)
        end_button.click(end_session, outputs=output_text)

    return sync_demo

Gradio Interface for Streaming Messages

In [59]:
def gradio_stream_ui():
    with gr.Blocks() as stream_demo:
        gr.Markdown("## SF Headless Agent - Stream Message")
        start_button = gr.Button("Start Session")
        input_text = gr.Textbox(label="Enter your message")
        stream_button = gr.Button("Send Stream Message")
        output_text = gr.Textbox(label="Stream Response")
        end_button = gr.Button("End Session")

        start_button.click(start_session, outputs=output_text)
        stream_button.click(send_message_stream, inputs=input_text, outputs=output_text)
        end_button.click(end_session, outputs=output_text)

    return stream_demo

Sync Message - Run Gradio apps in Colab

In [60]:
demo_sync = gradio_sync_ui()
demo_sync.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c48df7ed9d8169bb4b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Stream Message - Run Gradio apps in Colab

In [33]:
demo_sync = gradio_stream_ui()
demo_sync.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://481445352fc1743d25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
